In [1]:
#default_exp utils

In [3]:
#hide
from nbdev.showdoc import *

#%load_ext autoreload
#%autoreload 2

import sys
sys.path.insert(0, '..') #appends project root to path in order to import project packages since `noteboks_dev` is not on the root #appends project root to path in order to import project packages since `noteboks_dev` is not on the root


# Code

In [1]:
from scipy import stats

In [4]:
#export
from warnings import warn
from inspect import getmembers, isfunction
import inspect

import numpy as np
from scipy import sparse

from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

try:
    from sparse_dot_topn import awesome_cossim_topn
except Exception as e:
    warn(f'could not load sparse_dot_topn: {e}')
        

c:\users\user ambev\desktop\mypackages\heartwood\venv\lib\site-packages\ipykernel_launcher.py:15: UserWarning: could not load sparse_dot_topn: No module named 'sparse_dot_topn'
  from ipykernel import kernelapp as app


In [5]:
#export
#util funcs and classes


def get_default_args(func):
    '''THANKS TO mgilson at https://stackoverflow.com/questions/12627118/get-a-function-arguments-default-value'''
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }


def inherit_docstrings(cls):
    '''
    thanks to Martijn Pieters♦ at https://stackoverflow.com/questions/17393176/python-3-method-docstring-inheritance-without-breaking-decorators-or-violating
    '''
    for name, func in getmembers(cls, isfunction):
        if func.__doc__: continue
        for parent in cls.__mro__[1:]:
            if hasattr(parent, name):
                func.__doc__ = getattr(parent, name).__doc__
    return cls

In [6]:
# export
#TODO: implement minkowski distance with sparse_dot_topn
#TODO: implement RBF distance 

#export
def make_batches(arr, batch_size = 100):
    '''make batches for batch query'''
    #lst = [i for i in arr]

    if arr.shape[0] < batch_size:
        batches = [arr]
    else:
        n_bs = arr.shape[0] // batch_size
        last_batch = arr.shape[0] - batch_size * n_bs
        batches = []
        i = 0
        for i in range(n_bs):
            yield arr[i * batch_size:(i + 1) * batch_size]

        if last_batch:
            yield arr[(i + 1) * batch_size:]

def sim_matrix_to_idx_and_score(sim_matrix):
    '''
    returns list of indexes (col index of row vector) and scores (similarity value) for each row, given a similarity matrix
    '''
    scores = []
    idxs = []
    for row in sim_matrix:
        idxs.append(row.nonzero()[-1])
        scores.append(row.data)
    
    return idxs, scores

def cosine_similarity(A, B, topn = 30, remove_diagonal = False, **kwargs):        
    
    A,B = sparsify(A,B)
    A = normalize(A, norm  = 'l2').astype(np.float64)
    B = normalize(B, norm  = 'l2').astype(np.float64)
    dot = awesome_cossim_topn(A, B.T, ntop = topn, **kwargs)    
    
    if remove_diagonal:
        dot.setdiag(0)
        dot.eliminate_zeros()
    
    return dot


def cosine_distance(A, B, topn = 30, remove_diagonal = False, **kwargs):    
    
    #calculate sim
    dist = cosine_similarity(A, B, topn, remove_diagonal, **kwargs)
    #calculate distance
    dist.data = 1 - dist.data    
    return dist

In [ ]:
#export

def sparse_dot_product(
    A,
    B,
    ntop = 1,
    lower_bound=0,
    use_threads=False,
    n_jobs=1,
    return_best_ntop=False,
    test_nnz_max=-1,
):
    
    '''
    flexible dot product function to work with or without sparse_dot_topn. In the absence of sparse_dot_topn, naive numpy dot product will be performed
    
    sparse_dot_topn.awesome_cossim_topn Docs:
    
    This function will return a matrix C in CSR format, where
    C = [sorted top n results > lower_bound for each row of A * B].
    If return_best_ntop=True then best_ntop
    (the true maximum number of elements > lower_bound per row of A * B)
    will also be returned in a tuple together with C as (C, best_ntop).

    Input:
        A and B: two CSR matrices
        ntop: top n results
        lower_bound: a threshold that the element of A*B must be greater than
        use_threads: use multi-thread or not
        n_jobs: number of thread, must be >= 1
        return_best_ntop: (default: False) if True, will return best_ntop together 
                          with C as a tuple: (C, best_ntop)

    Output:
        C: result matrix (returned alone, if return_best_ntop=False)
        best_ntop: The true maximum number of elements > lower_bound per row of 
                   A * B returned together with C as a tuple: (C, best_ntop). It is 
                   returned only if return_best_ntop=True.

    N.B. if A and B are not in CSR format, they will be converted to CSR
    '''
    
    MAX_BYTES = 100e6 #process dense arrays of maximum 100MB for dense numpy dot product
    
    if 'awesome_cossim_topn' in globals():
        dot = awesome_cossim_topn(
            A = A,
            B = B,
            ntop = ntop,
            lower_bound=lower_bound,
            use_threads=use_threads,
            n_jobs=n_jobs,
            return_best_ntop=return_best_ntop,
            test_nnz_max=test_nnz_max,
        )
    else:
        warn('sparse_dot_topn is not installed, this may cause performance issues in dot product calculations')
        dot = A@B
    
    return dot

In [7]:
#export
def similarity_plot(vector, query_matrix):
    '''
    plots similarity plots like in https://gdmarmerola.github.io/forest-embeddings/
    '''
    return


In [8]:
#export
def sparsify(*arrs):
    '''
    makes input arrs sparse
    '''
    arrs = list(arrs)
    for i in range(len(arrs)):        
        if not sparse.issparse(arrs[i]):
            arrs[i] = sparse.csr_matrix(arrs[i])
    
    return arrs

def _robust_stack(blocks, stack_method = 'stack', **kwargs):
    
    if any(sparse.issparse(i) for i in blocks):
        #handle sparse
        stacked = getattr(sparse, stack_method)(blocks, **kwargs)
    
    else:        
        #handle pandas
        if all(hasattr(i, 'iloc') for i in blocks):
            if stack_method == 'hstack':                
                stacked = pd.concat(blocks, axis = 1)
            else:
                stacked = pd.concat(blocks, axis = 0)
        
        else:
            #handle  numpy
            stacked = getattr(np, stack_method)(blocks, **kwargs)
    
    return stacked

def hstack(blocks, **kwargs):
    return _robust_stack(blocks, stack_method = 'hstack', **kwargs)

def vstack(blocks, **kwargs):
    return _robust_stack(blocks, stack_method = 'vstack', **kwargs)

def stack(blocks, **kwargs):
    return _robust_stack(blocks, stack_method = 'stack', **kwargs)


class RobustEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self,):            
        '''
        A robust one hot encoder. Always return the same amount of nonzero value sin each transformed row.
        Has columns for unknown values
        '''
        return
    
    def fit(self, X, y = None, **kwawrgs):        
        self.ordinalencoder_ = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1).fit(X)
        
        X = self.ordinalencoder_.transform(X)
        
        categories = [np.arange(-1, len(cats)) for cats in self.ordinalencoder_.categories_]
        self.onehotencoder_ = OneHotEncoder(categories = categories).fit(X)        
        return self
    
    def transform(self, X, **kwargs):
        X = self.ordinalencoder_.transform(X)
        return self.onehotencoder_.transform(X)

In [9]:
#session of variables to testing sessions
n_features = 7
cardinality_per_feature = 1000
n_classes = 5
n_reg_dims = 2


X = np.random.randint(0,cardinality_per_feature,(1000,n_features))

y_class = np.random.randint(0,n_classes, 1000)
y_reg = np.random.randn(1000,n_reg_dims)

In [10]:
x = [['aaa'], ['bbb'], ['ccc'], ['ddd']]
enc = RobustEncoder().fit(x)

enc.transform([['aaa'],['asdasd'], ['asd'],['ccc']]).A

array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

# Export

In [4]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted cluster.ipynb.
Converted index.ipynb.
Converted kernel.ipynb.
Converted utils.ipynb.
